# __Presidents of the United States of America__

#### Let's scrape wikipedia: [List of presidents of the United States](https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States)

In [1]:
import pandas as pd

url = 'https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States'

# Grab all tables from the page
tables = pd.read_html(url)

# Inspect and select the correct table
presidents_df = tables[0]

# Preview the first few rows
presidents_df.head()

,No.[a],Portrait,Name (birth–death),Term[16],Party[b][17],Party[b][17].1,Election,Vice President[18]
0,1,NaN,George Washington (1732–1799) [19],"April 30, 1789 – March 4, 1797",NaN,Unaffiliated,1788–89 1792,John Adams[c]
1,2,NaN,John Adams (1735–1826) [21],"March 4, 1797 – March 4, 1801",NaN,Federalist,1796,Thomas Jefferson[d]
2,3,NaN,Thomas Jefferson (1743–1826) [23],"March 4, 1801 – March 4, 1809",NaN,Democratic- Republican,1800 1804,Aaron Burr George Clinton
3,4,NaN,James Madison (1751–1836) [24],"March 4, 1809 – March 4, 1817",NaN,Democratic- Republican,1808 1812,"George Clinton[e] Vacant after April 20, 1812 ..."
4,5,NaN,James Monroe (1758–1831) [26],"March 4, 1817 – March 4, 1825",NaN,Democratic- Republican,1816 1820,Daniel D. Tompkins


## __Data Cleaning__

#### Next, we will clean the dataset and then save it to a .CSV file for convenience.

__Notice:__

- `Portrait` is always `NaN`, so we can drop this column.

- There are 2 `Party` columns—the first one was for the party color. We can ignore this column too.

- We must clean up column names and row entires that have footnotes.

- We can enforce snake_case column names so we don't have to guess.

- We should also:

	- Split `Name (birth–death)` into 3 separate columns (`name`, `birth`, `death`)

	- Split `Term` into `term_start` and `term_end` for accessibility

	- Standardize the party names

	- Use consistent date formatting

In [2]:
# Copy original DataFrame to avoid altering it directly
df = presidents_df.copy()

# Drop the `Portrait` and first `Party` column
df.drop(df.columns[[1, 4]], axis=1, inplace=True)

# Rename columns
df.columns = [
	'number',
	'name_birth_death',
	'term',
	'party',
	'election',
	'vice_president'
]

# Let's set presidential number as the DataFrame index
df.set_index('number', inplace=True)

# Show our progress
df.head(10)

,name_birth_death,term,party,election,vice_president
number,,,,,
1,George Washington (1732–1799) [19],"April 30, 1789 – March 4, 1797",Unaffiliated,1788–89 1792,John Adams[c]
2,John Adams (1735–1826) [21],"March 4, 1797 – March 4, 1801",Federalist,1796,Thomas Jefferson[d]
3,Thomas Jefferson (1743–1826) [23],"March 4, 1801 – March 4, 1809",Democratic- Republican,1800 1804,Aaron Burr George Clinton
4,James Madison (1751–1836) [24],"March 4, 1809 – March 4, 1817",Democratic- Republican,1808 1812,"George Clinton[e] Vacant after April 20, 1812 ..."
5,James Monroe (1758–1831) [26],"March 4, 1817 – March 4, 1825",Democratic- Republican,1816 1820,Daniel D. Tompkins
6,John Quincy Adams (1767–1848) [27],"March 4, 1825 – March 4, 1829",Democratic- Republican[f] National Republican,1824,John C. Calhoun[g]
7,Andrew Jackson (1767–1845) [30],"March 4, 1829 – March 4, 1837",Democratic,1828 1832,"John C. Calhoun[h] Vacant after December 28, 1..."
8,Martin Van Buren (1782–1862) [31],"March 4, 1837 – March 4, 1841",Democratic,1836,Richard Mentor Johnson
9,William Henry Harrison (1773–1841) [32],"March 4, 1841 – April 4, 1841[e]",Whig,1840,John Tyler


In [3]:
import re

# Remove footnotes like [19] or [c]
def remove_footnotes(text):
	if not isinstance(text, str):
		return text
	return re.sub(r'\[.*?\]', '', text).strip()

# Apply function to each value in a column
df['name_birth_death'] = df['name_birth_death'].apply(remove_footnotes)
df['vice_president'] = df['vice_president'].apply(remove_footnotes)
df['party'] = df['party'].apply(remove_footnotes)
df['term'] = df['term'].apply(remove_footnotes)

# See progress
df.head(10)

,name_birth_death,term,party,election,vice_president
number,,,,,
1,George Washington (1732–1799),"April 30, 1789 – March 4, 1797",Unaffiliated,1788–89 1792,John Adams
2,John Adams (1735–1826),"March 4, 1797 – March 4, 1801",Federalist,1796,Thomas Jefferson
3,Thomas Jefferson (1743–1826),"March 4, 1801 – March 4, 1809",Democratic- Republican,1800 1804,Aaron Burr George Clinton
4,James Madison (1751–1836),"March 4, 1809 – March 4, 1817",Democratic- Republican,1808 1812,"George Clinton Vacant after April 20, 1812 Elb..."
5,James Monroe (1758–1831),"March 4, 1817 – March 4, 1825",Democratic- Republican,1816 1820,Daniel D. Tompkins
6,John Quincy Adams (1767–1848),"March 4, 1825 – March 4, 1829",Democratic- Republican National Republican,1824,John C. Calhoun
7,Andrew Jackson (1767–1845),"March 4, 1829 – March 4, 1837",Democratic,1828 1832,"John C. Calhoun Vacant after December 28, 1832..."
8,Martin Van Buren (1782–1862),"March 4, 1837 – March 4, 1841",Democratic,1836,Richard Mentor Johnson
9,William Henry Harrison (1773–1841),"March 4, 1841 – April 4, 1841",Whig,1840,John Tyler


In [4]:
# Next, we will clean the party names

# Let's see all unique values in this column
for unique_party in df['party'].unique():
	print(unique_party)

Unaffiliated
Federalist
Democratic- Republican
Democratic- Republican National Republican
Democratic
Whig
Whig Unaffiliated
Republican National Union
National Union Democratic
Republican


In [5]:
recognized_parties = set([
	'Unaffiliated',
	'Federalist',
	'Democratic-Republican',
	'National Republican',
	'Democratic',
	'Whig',
	'National Union',
	'Republican'
])

def extract_parties(text):
	''' Return a list of recognized parties from `text` '''
	if not isinstance(text, str):
		return text
	
	# Get rid of spaces next to dashes
	text = re.sub(r'\s*-\s*', '-', text)

	# Remove extra spaces and enforce Title Case
	text = re.sub(r'\s+', ' ', text).strip().title()

	# Find all recognized parties
	parts = text.split()

	found = []
	current = ''
	for part in parts:
		# Continue building
		if current:
			current += ' ' + part
		else:
			current = part
		# Found party
		if current in recognized_parties:
			found.append(current)
			current = ''
	if current in recognized_parties:
		found.append(current)

	return found

df['party'] = df['party'].apply(extract_parties)

# Show parties
df[['party']].head(10)

,party
number,
1,[Unaffiliated]
2,[Federalist]
3,[Democratic-Republican]
4,[Democratic-Republican]
5,[Democratic-Republican]
6,"[Democratic-Republican, National Republican]"
7,[Democratic]
8,[Democratic]
9,[Whig]


Now, let's work on extracting the election dates and vice presidents into lists as well.

Keep in mind, vice president entries may contain multiple names or vacancy messages.

There are 3 tokens we must look out for:

1. VP name

2. "Vacant throughout presidency" (verbatim)

3. Vacant message of the form:

	`Vacant` + `...` + `year`

__Ex.__

> John C. Calhoun Vacant after December 28, 1832 Martin Van Buren

- (1) John C. Calhoun

- (3) Vacant after December 28, 1832

- (1) Martin Van Buren

In [6]:
# Show examples in the vice president column
for i in range(1, 31):
	print(f"{i}: {df.loc[i, 'vice_president']}")

1: John Adams
2: Thomas Jefferson
3: Aaron Burr George Clinton
4: George Clinton Vacant after April 20, 1812 Elbridge Gerry Vacant after November 23, 1814
5: Daniel D. Tompkins
6: John C. Calhoun
7: John C. Calhoun Vacant after December 28, 1832 Martin Van Buren
8: Richard Mentor Johnson
9: John Tyler
10: Vacant throughout presidency
11: George M. Dallas
12: Millard Fillmore
13: Vacant throughout presidency
14: William R. King Vacant after April 18, 1853
15: John C. Breckinridge
16: Hannibal Hamlin Andrew Johnson
17: Vacant throughout presidency
18: Schuyler Colfax Henry Wilson Vacant after November 22, 1875
19: William A. Wheeler
20: Chester A. Arthur
21: Vacant throughout presidency
22: Thomas A. Hendricks Vacant after November 25, 1885
23: Levi P. Morton
24: Adlai Stevenson I
25: Garret Hobart Vacant after November 21, 1899 Theodore Roosevelt
26: Vacant through March 4, 1905 Charles W. Fairbanks
27: James S. Sherman Vacant after October 30, 1912
28: Thomas R. Marshall
29: Calvin Coo

In [7]:
def extract_vice_presidents(text):
	if not isinstance(text, str):
		return text

	# Replace &nbsp; and normalize whitespace
	text = text.replace('\xa0', ' ')
	text = re.sub(r'\s+', ' ', text).strip()

	# Split the string at each vacancy message, keeping the vacancy tokens
	split_text = re.split(r'(Vacant throughout presidency|Vacant.*?\d{4})', text, flags=re.IGNORECASE)

	return list(token.strip() for token in split_text if token)

df['vice_president'] = df['vice_president'].apply(extract_vice_presidents)

# Show vice presidents
df[['vice_president']].head(10)

,vice_president
number,
1,[John Adams]
2,[Thomas Jefferson]
3,[Aaron Burr George Clinton]
4,"[George Clinton, Vacant after April 20, 1812, ..."
5,[Daniel D. Tompkins]
6,[John C. Calhoun]
7,"[John C. Calhoun, Vacant after December 28, 18..."
8,[Richard Mentor Johnson]
9,[John Tyler]


In [8]:
def extract_election_dates(text):
	if not isinstance(text, str):
		return text
	years = []
	for s in text.split():
		if '–' in s: # Ignore year ranges and only take the first (ex. 1788–89)
			s = s.split('–')[0]
		try:
			years.append(int(s))
		except Exception:
			years.append(pd.NA)
	return years

df['election'] = df['election'].apply(extract_election_dates)

# Show election column
df[['election']].head()

,election
number,
1,"[1788, 1792]"
2,[1796]
3,"[1800, 1804]"
4,"[1808, 1812]"
5,"[1816, 1820]"


Perfect!

Next, we will split the `name_birth_death` and `term` columns into multiple data points.

In [9]:
def split_name_birth_death(text):
	match = re.match(r'(.*?)\s*\((?:(\d{4})–(\d{4})|b\. (\d{4}))\)', text)
	if not match:
		return pd.Series([pd.NA] * 3)
	
	# Extract from capture groups
	name = match.group(1)
	birth = match.group(2) if match.group(2) else match.group(4)
	death = match.group(3) if match.group(3) else pd.NA

	return pd.Series([name, birth, death])

# We will add 3 new columns
df[['name', 'birth', 'death']] = df['name_birth_death'].apply(split_name_birth_death)

# Add the new columns to the front
columns = df.columns.tolist()
columns = columns[-3:] + columns[:-3]
df = df[columns]

df.tail(10)

,name,birth,death,name_birth_death,term,party,election,vice_president
number,,,,,,,,
38,Gerald Ford,1913,2006,Gerald Ford (1913–2006),"August 9, 1974 – January 20, 1977",[Republican],[<NA>],"[Vacant through December 19, 1974, Nelson Rock..."
39,Jimmy Carter,1924,2024,Jimmy Carter (1924–2024),"January 20, 1977 – January 20, 1981",[Democratic],[1976],[Walter Mondale]
40,Ronald Reagan,1911,2004,Ronald Reagan (1911–2004),"January 20, 1981 – January 20, 1989",[Republican],"[1980, 1984]",[George H. W. Bush]
41,George H. W. Bush,1924,2018,George H. W. Bush (1924–2018),"January 20, 1989 – January 20, 1993",[Republican],[1988],[Dan Quayle]
42,Bill Clinton,1946,<NA>,Bill Clinton (b. 1946),"January 20, 1993 – January 20, 2001",[Democratic],"[1992, 1996]",[Al Gore]
43,George W. Bush,1946,<NA>,George W. Bush (b. 1946),"January 20, 2001 – January 20, 2009",[Republican],"[2000, 2004]",[Dick Cheney]
44,Barack Obama,1961,<NA>,Barack Obama (b. 1961),"January 20, 2009 – January 20, 2017",[Democratic],"[2008, 2012]",[Joe Biden]
45,Donald Trump,1946,<NA>,Donald Trump (b. 1946),"January 20, 2017 – January 20, 2021",[Republican],[2016],[Mike Pence]
46,Joe Biden,1942,<NA>,Joe Biden (b. 1942),"January 20, 2021 – January 20, 2025",[Democratic],[2020],[Kamala Harris]


In [10]:
def split_term(text):
	terms = re.split(r'\s*–\s*', text)
	if len(terms) == 2:
		start = terms[0].strip()
		end = terms[1].strip()
		if end == 'Incumbent': # Term not over
			end = pd.NA
		return pd.Series([start, end])
	# Failed to extract 2 tokens
	return pd.Series([pd.NA] * 2)

df[['term_start', 'term_end']] = df['term'].apply(split_term)

columns = df.columns.tolist()
columns = columns[:4] + columns[-2:] + columns[4:-2]
df = df[columns]

df.tail(10)

,name,birth,death,name_birth_death,term_start,term_end,term,party,election,vice_president
number,,,,,,,,,,
38,Gerald Ford,1913,2006,Gerald Ford (1913–2006),"August 9, 1974","January 20, 1977","August 9, 1974 – January 20, 1977",[Republican],[<NA>],"[Vacant through December 19, 1974, Nelson Rock..."
39,Jimmy Carter,1924,2024,Jimmy Carter (1924–2024),"January 20, 1977","January 20, 1981","January 20, 1977 – January 20, 1981",[Democratic],[1976],[Walter Mondale]
40,Ronald Reagan,1911,2004,Ronald Reagan (1911–2004),"January 20, 1981","January 20, 1989","January 20, 1981 – January 20, 1989",[Republican],"[1980, 1984]",[George H. W. Bush]
41,George H. W. Bush,1924,2018,George H. W. Bush (1924–2018),"January 20, 1989","January 20, 1993","January 20, 1989 – January 20, 1993",[Republican],[1988],[Dan Quayle]
42,Bill Clinton,1946,<NA>,Bill Clinton (b. 1946),"January 20, 1993","January 20, 2001","January 20, 1993 – January 20, 2001",[Democratic],"[1992, 1996]",[Al Gore]
43,George W. Bush,1946,<NA>,George W. Bush (b. 1946),"January 20, 2001","January 20, 2009","January 20, 2001 – January 20, 2009",[Republican],"[2000, 2004]",[Dick Cheney]
44,Barack Obama,1961,<NA>,Barack Obama (b. 1961),"January 20, 2009","January 20, 2017","January 20, 2009 – January 20, 2017",[Democratic],"[2008, 2012]",[Joe Biden]
45,Donald Trump,1946,<NA>,Donald Trump (b. 1946),"January 20, 2017","January 20, 2021","January 20, 2017 – January 20, 2021",[Republican],[2016],[Mike Pence]
46,Joe Biden,1942,<NA>,Joe Biden (b. 1942),"January 20, 2021","January 20, 2025","January 20, 2021 – January 20, 2025",[Democratic],[2020],[Kamala Harris]


In [11]:
# Now we can drop the old columns we no longer need
df.drop(columns=['name_birth_death', 'term'], inplace=True)
df.head()

,name,birth,death,term_start,term_end,party,election,vice_president
number,,,,,,,,
1,George Washington,1732,1799,"April 30, 1789","March 4, 1797",[Unaffiliated],"[1788, 1792]",[John Adams]
2,John Adams,1735,1826,"March 4, 1797","March 4, 1801",[Federalist],[1796],[Thomas Jefferson]
3,Thomas Jefferson,1743,1826,"March 4, 1801","March 4, 1809",[Democratic-Republican],"[1800, 1804]",[Aaron Burr George Clinton]
4,James Madison,1751,1836,"March 4, 1809","March 4, 1817",[Democratic-Republican],"[1808, 1812]","[George Clinton, Vacant after April 20, 1812, ..."
5,James Monroe,1758,1831,"March 4, 1817","March 4, 1825",[Democratic-Republican],"[1816, 1820]",[Daniel D. Tompkins]


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47 entries, 1 to 47
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            47 non-null     object
 1   birth           47 non-null     object
 2   death           41 non-null     object
 3   term_start      47 non-null     object
 4   term_end        46 non-null     object
 5   party           47 non-null     object
 6   election        47 non-null     object
 7   vice_president  47 non-null     object
dtypes: object(8)
memory usage: 4.4+ KB


Finally, let's convert some of our columns to use pandas datetime dtype.

This allows for:

- Efficient date comparisons

- Extracting (month, day, year)

- Date arithmetic

- Easy conversion between date formats for saving or display

In [13]:
df['name'] = df['name'].astype('string')

# Convert birth and death columns to nullable integers (supports pd.NA)
df['birth'] = df['birth'].astype('Int64')
df['death'] = df['death'].astype('Int64')

# Convert string columns to datetime where appropriate
df['term_start'] = pd.to_datetime(df['term_start'], errors='coerce')
df['term_end'] = pd.to_datetime(df['term_end'], errors='coerce')

# We'll leave `election` as a list of strings to account for the 1788–1789 election
# The user can deal with this after loading the dataset

# Confirm the changes
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47 entries, 1 to 47
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   name            47 non-null     string        
 1   birth           47 non-null     Int64         
 2   death           41 non-null     Int64         
 3   term_start      47 non-null     datetime64[ns]
 4   term_end        46 non-null     datetime64[ns]
 5   party           47 non-null     object        
 6   election        47 non-null     object        
 7   vice_president  47 non-null     object        
dtypes: Int64(2), datetime64[ns](2), object(3), string(1)
memory usage: 4.4+ KB


## __Saving The Dataset__

We now neatly store all the data want to have access to!

Let's explore how we can save and retrieve this data in the future, preserving our data types.

In [14]:
# We will convert list objects to pipe-separated strings 
# to preserve their structure in a human-readable way
def list_to_pipe_string(lst):
	''' Convert a list to a pipe-separated string, replacing `pd.NA` with "NA". '''
	if not lst:
		return pd.NA
	if isinstance(lst, list):
		return ' | '.join('NA' if pd.isna(item) else str(item) for item in lst)
	return lst

def pipe_string_to_list(s):
	''' Convert a pipe-separated string back to a list, converting "NA" to `pd.NA`. '''
	if pd.isna(s):
		return [pd.NA]
	if isinstance(s, str):
		return [pd.NA if item.strip() == 'NA' else item.strip() for item in s.split('|')]
	return s


# Column groupings
nullable_int_cols = ['birth', 'death']
datetime_cols = ['term_start', 'term_end']
list_cols = ['party', 'election', 'vice_president']

def save_df_to_csv(df, filename):
	''' Prepares and saves the DataFrame to a CSV file without modifying the original.
		- saves as `filename`.csv
	'''
	df_temp = df.copy()

	# Format datetime columns as strings (Month Day, Year)
	for col in datetime_cols:
		df_temp[col] = df_temp[col].apply(
			lambda d: f'{d.month_name()} {d.day}, {d.year}' if pd.notna(d) else pd.NaT
		)
	
	# Serialize list columns to JSON strings
	for col in list_cols:
		df_temp[col] = df_temp[col].apply(list_to_pipe_string)
	
	# Represent null values as NA
	# Use UTF-8 BOM so Excel can correctly display Unicode characters
	df_temp.to_csv(f'{filename}.csv', na_rep='NA', encoding='utf-8-sig')

def load_df_from_csv(filepath):
	''' Loads and restores the DataFrame from a saved CSV file. '''
	df = pd.read_csv(filepath, index_col=0, na_values='NA')

	# Convert datetime strings back to datetime objects
	for col in datetime_cols:
		df[col] = pd.to_datetime(df[col], errors='coerce')
	
	# Restore lists from JSON
	for col in list_cols:
		df[col] = df[col].apply(pipe_string_to_list)

	# Restore nullable integer columns
	for col in nullable_int_cols:
		df[col] = df[col].astype('Int64')

	# Restore election list to integers
	df['election'] = df['election'].apply(lambda lst: [pd.NA if pd.isna(x) else int(x) for x in lst])

	# Convert name column to strings
	df['name'] = df['name'].astype('string')
	
	return df

#### We can finally save the cleaned dataset to .CSV

In [15]:
save_df_to_csv(df, 'us_presidents_cleaned')

#### Let's try loading the data back into a pandas DataFrame!

In [16]:
df_loaded = load_df_from_csv('us_presidents_cleaned.csv')

In [17]:
# Let's make sure df_loaded is the same as our original df:

# Same number of rows/cols
print(df.shape == df_loaded.shape)

# Same column names
print(df.columns.equals(df_loaded.columns))

# Same indices
print(df.index.equals(df_loaded.index))

# Same data
df.equals(df_loaded)

True
True
True


True

In [18]:
df_loaded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47 entries, 1 to 47
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   name            47 non-null     string        
 1   birth           47 non-null     Int64         
 2   death           41 non-null     Int64         
 3   term_start      47 non-null     datetime64[ns]
 4   term_end        46 non-null     datetime64[ns]
 5   party           47 non-null     object        
 6   election        47 non-null     object        
 7   vice_president  47 non-null     object        
dtypes: Int64(2), datetime64[ns](2), object(3), string(1)
memory usage: 3.4+ KB


In [19]:
df_loaded.tail()

,name,birth,death,term_start,term_end,party,election,vice_president
number,,,,,,,,
43,George W. Bush,1946,<NA>,2001-01-20,2009-01-20,[Republican],"[2000, 2004]",[Dick Cheney]
44,Barack Obama,1961,<NA>,2009-01-20,2017-01-20,[Democratic],"[2008, 2012]",[Joe Biden]
45,Donald Trump,1946,<NA>,2017-01-20,2021-01-20,[Republican],[2016],[Mike Pence]
46,Joe Biden,1942,<NA>,2021-01-20,2025-01-20,[Democratic],[2020],[Kamala Harris]
47,Donald Trump,1946,<NA>,2025-01-20,NaT,[Republican],[2024],[JD Vance]
